In [1]:
# importing required libraries

import pandas as pd 
import numpy as np
import altair as alt

In [2]:
# importing the data 

df = pd.read_csv('minard-data.csv')

In [3]:
# modifying data to create map with positions of the city
modify = df.dropna(subset=['CITY'])
city = modify[['LONC', 'LATC', 'CITY']]

# creating map with city names and markings using altair library
cityMap = alt.Chart(city).mark_circle(size=65, color="black").encode(
        longitude='LONC:Q', latitude='LATC:Q',
        tooltip=[alt.Tooltip('LATC', title='Latitude'), alt.Tooltip('LONC', title='Longitude'),
                 alt.Tooltip('CITY', title='City')]
    ) + alt.Chart(city).mark_text(font='Comic Sans MS', fontSize=12, dx=7, dy=-12).encode(
    longitude='LONC:Q', latitude='LATC:Q', text='CITY')

In [4]:
# modifying and sorting data for the path of the army
army = df[['LONP','LATP','SURV','DIR','DIV']]
army = army.sort_values(by=["DIV", "SURV"], ascending=False)

armyText = army.iloc[::2, :].copy()
armyText["LONP"] += 0.13 * (armyText["DIV"])
armyText["LATP"] += armyText["DIR"].replace({"A": 0.35, "R": -0.21})

# creating map with path of the army using altair library
armyMap = alt.Chart(army).mark_trail(opacity=0.5).encode(
    longitude='LONP',latitude='LATP', size=alt.Size('SURV',scale=alt.Scale(range=[10, 50]),
                legend = alt.Legend(title = 'Survivors', symbolSize = 300, titleFontSize = 15, labelFontSize = 15)),
    detail='DIR',color=alt.Color(
        'DIR',scale=alt.Scale(
            domain=['A', 'R'],
            range=['purple', 'orange']),
        legend = alt.Legend(title = 'Direction', symbolSize = 1000, titleFontSize = 15, labelFontSize = 15)),
    tooltip=[alt.Tooltip('LATP', title='Latitude'), alt.Tooltip('LONP', title='Longitude'), 
             alt.Tooltip('SURV', title='Survivors'), alt.Tooltip('DIR', title='Direction')]
) + alt.Chart(armyText).mark_text(font='Comic Sans MS', fontSize=8, fontStyle='italic', dx=1, dy=8, angle=280).encode(
    longitude='LONP:Q', latitude='LATP:Q', text='SURV:Q') + alt.Chart(army).mark_trail(opacity=0.5).encode(
    longitude='LONP:Q',latitude='LATP:Q',
    detail='DIV',color=alt.Color(
        'DIV',scale=alt.Scale(
            domain=['1', '2', '3'],
            range=['red', 'green', 'blue']), 
        legend = alt.Legend(title = 'Division', direction = 'horizontal', symbolSize = 1000, titleFontSize = 15, 
                            labelFontSize = 15))).properties(height=400, width=1000)

In [5]:
# Combining maps of city markings and path of army using altair library
# You can find the number of survivors when you hover over the path of army 
X = alt.X(
    'LONP', scale=alt.Scale(domain=[army["LONP"].min(), army["LONP"].max()]),
    axis=alt.Axis(title="Longitude", grid=True, titleFontSize = 20, titleColor='gray'))

Y = alt.Y(
    'LATP', scale=alt.Scale(domain=[army["LATP"].min() - 1.25, army["LATP"].max() + 1.25]),
    axis=alt.Axis(title="Latitude", grid=True, orient="left", titleFontSize = 20, titleColor='gray'))
    
maps = cityMap + armyMap + alt.Chart(army).mark_text().encode(
    X, Y). properties(height=400, width=1000, title = "Minard’s visualization of Napolean’s Russian Campaign")


#Uncomment the following comment to get 
# (a) position of cities and path of army through them
# (b) survivors in the army along the path
#maps

In [6]:
# Modifying data to create map for temperature during retreat
modDf = df.dropna(subset=['TEMP'])
temp = modDf[['LONT','TEMP','DAYS','MON', 'DAY']]
pd.set_option('mode.chained_assignment', None)
temp['DAYS'] = temp['DAYS'].astype('str').str.strip('.0')
temp['MON'] = temp['MON'].astype('str')
temp = temp.replace('nan', np.nan)
temp["date"] = temp.fillna("").apply(axis=1, func=lambda row: "{}° {} {}".format(row[1], row[3], row[2]))

# creating map for temperature during retreat using altair library
x = alt.X(
        'LONT', scale=alt.Scale(domain=[army["LONP"].min(), army["LONP"].max()]),
        axis=alt.Axis(title="Longitude", grid=True, titleFontSize = 20, titleColor='gray'))

y = alt.Y(
        'TEMP', axis=alt.Axis(title="Temperature on Retreat", grid=True, orient='right', titleFontSize = 20, titleColor='gray'),
        scale=alt.Scale(domain=[temp["TEMP"].min() - 10, temp["TEMP"].max() + 10]))

tempMap = alt.Chart(temp).mark_line(
    color="orange").encode(
    x, y,) + alt.Chart(temp).mark_circle(size=50, color="blue").encode(
    x,y,tooltip=[alt.Tooltip('LONT', title='Longitude'), alt.Tooltip('TEMP', title='Temperature'), 
                 alt.Tooltip('date', title='Date')]
    ) + alt.Chart(temp).mark_text(dx=5, dy=20, font='Comic Sans MS', fontSize=12).encode(
    x, y, text='date').properties(height=400, width=1000, title = 'Temperature in degrees during retreat')

#Uncomment the following comment to get 
# (c) temperature during retreat
#tempMap

In [7]:
# Concatinating the maps to create one final map
finalMap = alt.vconcat(maps, tempMap).configure_view(
        width=2000, height=1500, strokeWidth=0).configure_axis(
    grid=False, labelFont="Comic Sans MS", titleFont="Comic Sans MS").configure_title(
    fontSize=30, font='Comic Sans MS', color='black')

In [8]:
finalMap

alt.VConcatChart(...)